In [18]:
import random
import pandas as pd

object_class = "chair"
object_class_shapnet_id = "03001627"
num_train = 2

df = pd.read_csv('/scratch/noam/shapetalk/language/shapetalk_preprocessed_public_version_0.csv')
df = df[df["source_object_class"] == object_class]
wnlemmas_df = pd.read_csv(f"/scratch/noam/ShapeNetCore.v1/{object_class_shapnet_id}.csv")

def find_wnlemma(target_uid):
    l = wnlemmas_df[wnlemmas_df["fullId"] == f"3dw.{target_uid}"]
    if len(l) == 0:
        return object_class
    wnlemmas = l.iloc[0]["wnlemmas"].split(",")
    if len(wnlemmas) == 1:
        return wnlemmas[0]
    if object_class in wnlemmas:
        wnlemmas.remove(object_class)
    return random.choice(wnlemmas)

df["target_object_wnlemma"] = df["target_uid"].apply(lambda x: find_wnlemma(x.split("/")[-1]))

def func1(wnlemma, utterance):
    print('\n')
    print(wnlemma)
    print(utterance)
    return input()


for i, row in df.iterrows():
    if i == num_train:
        break
    df.loc[i, "llama3_target_utterance"] = func1(row["target_object_wnlemma"], row["utterance"])
df.to_csv(f"{object_class}.csv")



garden chair
Target chair is smaller and lower to the ground.


lounge
The target chair is part of a row of waiting chairs.


chair
The target has fewer spindles than the distractor.


chair
It is shorter


side chair
It has spindles


chair
The target chairs' back has a large trapezoid cutout.


chair
The target chair is made of wire.


straight chair
It has a pillow on it


armchair
the back legs are a flat piece


chair
It has a top rail.


side chair
the seat is thicker


chair
The chair legs are narrower


armchair
The target chair has shorter legs.


chair
The target has shorter legs.


chair
The back rest is longer and higher.


chair
It has two legs.


chaise
The target is foldable.


armchair
Its seat surface is much thinner.


chair
It does not have roller wheels.


chair
The target chair has no seat cushion.


chair
the legs are short


chair
Target chair is longer and slimmer.


straight chair
it is not a lounge chair


chair
The seat sits more level


chair
The seat is m

KeyboardInterrupt: Interrupted by user

In [21]:
columns = ["instruction", "input", "output"]
rows = []
for i, row in df.dropna(subset=['llama3_target_utterance']).iterrows():
    instruction = f"You will be given an utterance comparing {row['target_object_wnlemma']} to another {object_class}. Your task is to rewrite the given utterance as a description specifically about the {row['target_object_wnlemma']}."
    input = f"Can you rewrite please the following utterance comparing {row['target_object_wnlemma']} to another {object_class} as a description specifically about the {row['target_object_wnlemma']}? Utterance: {row['utterance']}."
    output = row["llama3_target_utterance"]
    rows.append([instruction, input, output])
pd.DataFrame(rows, columns=columns).to_csv(f"{object_class}_train.csv", index=False)

To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://github.com/unslothai/unsloth#installation-instructions---conda).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save) (eg for Llama.cpp).

**[NEW] Llama-3 8b is trained on a crazy 15 trillion tokens! Llama-2 was 2 trillion.**

In [17]:
%%capture
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


* We support Llama, Mistral, CodeLlama, TinyLlama, Vicuna, Open Hermes etc
* And Yi, Qwen ([llamafied](https://huggingface.co/models?sort=trending&search=qwen+llama)), Deepseek, all Llama, Mistral derived archs.
* We support 16bit LoRA or 4bit QLoRA. Both 2x faster.
* `max_seq_length` can be set to anything, since we do automatic RoPE Scaling via [kaiokendev's](https://kaiokendev.github.io/til) method.
* [**NEW**] With [PR 26037](https://github.com/huggingface/transformers/pull/26037), we support downloading 4bit models **4x faster**! [Our repo](https://huggingface.co/unsloth) has Llama, Mistral 4bit models.

In [18]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.679 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.25.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [19]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [20]:
from datasets import Dataset
import pyarrow.csv as pcsv

class MyDataset(Dataset):
    def __init__(
        self,
        csv_path: str,
    ):
      arrow_table = pcsv.read_csv(csv_path)
      super().__init__(arrow_table=arrow_table)

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `ChatML` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing).

For text completions like novel writing, try this [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing).

In [21]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = MyDataset(f'chair_train.csv')
dataset = dataset.map(formatting_prompts_func, batched = True,)
for k, v in dataset[0].items():
    print('@', k, ': ', v)

Map:   0%|          | 0/225 [00:00<?, ? examples/s]

@ instruction :  You will be given an utterance comparing garden chair to another chair. Your task is to rewrite the given utterance as a description specifically about the garden chair.
@ input :  Can you rewrite please the following utterance comparing garden chair to another chair as a description specifically about the garden chair? Utterance: Target chair is smaller and lower to the ground..
@ output :  a small garden chair
@ text :  Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
You will be given an utterance comparing garden chair to another chair. Your task is to rewrite the given utterance as a description specifically about the garden chair.

### Input:
Can you rewrite please the following utterance comparing garden chair to another chair as a description specifically about the garden chair? Utterance: Target chair is smaller and lower to the ground

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [22]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map (num_proc=2):   0%|          | 0/225 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [23]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA RTX A5000. Max memory = 23.679 GB.
16.762 GB of memory reserved.


In [24]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 225 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,2.838000
2,2.853900
3,2.808100
4,2.722100
5,2.397000
6,1.916600
7,1.442400
8,1.105800
9,0.741200
10,0.495800


In [25]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

89.6096 seconds used for training.
1.49 minutes used for training.
Peak reserved memory = 16.762 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 70.788 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!

In [11]:
# alpaca_prompt = Copied from above
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         f"Use only one of the following options, if none of them is suitable return Unknown. Options: {', '.join(options)}.", # instruction
#         f"Can you rewrite please the following description of a {obj}? Description: It is shorter.", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nUse only one of the following options, if none of them is suitable return Unknown. Options: a chair with a spindle backrest, a chair with a solid backrest, a chair with a thin seat, a chair with a thick seat, a chair with a narrow seat, a chair with a wide seat, a chair with a rounded backrest, a chair with a squared backrest, a chair with a short backrest, a chair with a long backrest, a chair with a curved backrest, a chair with a straight backrest, a chair without a legs strecher, a chair  with a legs strecher, a chair with short legs, a chair with long legs, a chair with thin legs, a chair with thick legs, a chair without armrests, a chair with armrests, a chair with two legs, a chair with four legs.\n\n### Input:\nCan you rewrite please the following description of a chair? Description:

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [20]:
# alpaca_prompt = Copied from above
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     alpaca_prompt.format(
#         f"Use only one of the following options, if none of them is suitable return Unknown. Options: {', '.join(options)}.", # instruction
#         f"Can you rewrite please the following description of a {obj}? Description: has thicker arm rest", # input
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Use only one of the following options, if none of them is suitable return Unknown. Options: a chair with a spindle backrest, a chair with a solid backrest, a chair with a thin seat, a chair with a thick seat, a chair with a narrow seat, a chair with a wide seat, a chair with a rounded backrest, a chair with a squared backrest, a chair with a short backrest, a chair with a long backrest, a chair with a curved backrest, a chair with a straight backrest, a chair without a legs strecher, a chair  with a legs strecher, a chair with short legs, a chair with long legs, a chair with thin legs, a chair with thick legs, a chair without armrests, a chair with armrests, a chair with two legs, a chair with four legs.

### Input:
Can you rewrite please the following description of a chair? Description: has thi

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [26]:
model.save_pretrained(f"/scratch/noam/llama3/chair/lora_model") # Local saving
tokenizer.save_pretrained(f"/scratch/noam/llama3/chair/lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('/scratch/noam/llama3/chair/lora_model/tokenizer_config.json',
 '/scratch/noam/llama3/chair/lora_model/special_tokens_map.json',
 '/scratch/noam/llama3/chair/lora_model/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [44]:
torch.cuda.empty_cache()

# Collect garbage
import gc
gc.collect()
import tqdm
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
instruct = "You will be given an utterance comparing {} to another chair. Your task is to rewrite the given utterance as a description specifically about the {}."
input = "Can you rewrite please the following utterance comparing {} to another chair as a description specifically about the {}? Utterance: {}."

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = f"/scratch/noam/llama3/chair/lora_model", 
    max_seq_length = 2048,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model)
csv_path = f'chair.csv'
df = pd.read_csv(csv_path)
for i, row in tqdm.tqdm(df.iterrows(), total=len(df)):
    if pd.notna(row["llama3_target_utterance"]):
        continue
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                instruct.format(row["target_object_wnlemma"], row["target_object_wnlemma"]),
                input.format(row["target_object_wnlemma"], row["target_object_wnlemma"], row["utterance"]),
                "",
            )
        ], return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    llama3_uttarance = tokenizer.batch_decode(outputs)[0].split("\n")[-1].replace(tokenizer.eos_token, "")
    if i % 100 == 0:
        print(row["target_object_wnlemma"], '+', row["utterance"], ' -> ', llama3_uttarance)
    df.at[i, 'llama3_target_utterance'] = llama3_uttarance
    if i == 1000:
        break
df.to_csv(csv_path, index=False)


==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA RTX A5000. Max memory: 23.679 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.25.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/tmp/ipykernel_2855247/3333724864.py:28: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)
  1%|▏         | 1087/74812 [00:30<1:12:04, 17.05it/s]

chair + The target has thin and long legs  ->  a chair with thin and long legs


  2%|▏         | 1197/74812 [01:08<5:02:57,  4.05it/s]

straight chair + There are no legs attached to the chair.  ->  a straight chair without legs attached to the chair


...

chaise + Target has elongated seating  ->  a chaise with an elongated seating


 20%|██        | 15001/74812 [1:21:06<5:55:56,  2.80it/s]

side chair + The back rest is flatter and less curved.  ->  a side chair with a flat and less curved back rest


 20%|██        | 15040/74812 [1:21:19<5:23:11,  3.08it/s]


KeyboardInterrupt: 

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is a famous tall tower in Paris?\n\n### Input:\n\n\n### Response:\nThe Eiffel Tower is a famous tall tower in Paris, France. It is 324 meters (1,063 feet) tall and was built in 1889 for the World's Fair. It is the most visited paid monument in the world, with over 7 million visitors annually. The tower is named after"]

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in `llama.cpp` or a UI based system like `GPT4All`. You can install GPT4All by going [here](https://gpt4all.io/index.html).

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/u54VK8m8tk) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Zephyr DPO 2x faster [free Colab](https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing)
2. Llama 7b 2x faster [free Colab](https://colab.research.google.com/drive/1lBzz5KeZJKXjvivbYvmGarix9Ao6Wxe5?usp=sharing)
3. TinyLlama 4x faster full Alpaca 52K in 1 hour [free Colab](https://colab.research.google.com/drive/1AZghoNBQaMDgWJpi4RbffGM1h6raLUj9?usp=sharing)
4. CodeLlama 34b 2x faster [A100 on Colab](https://colab.research.google.com/drive/1y7A0AxE3y8gdj4AVkl2aZX47Xu3P1wJT?usp=sharing)
5. Mistral 7b [free Kaggle version](https://www.kaggle.com/code/danielhanchen/kaggle-mistral-7b-unsloth-notebook)
6. We also did a [blog](https://huggingface.co/blog/unsloth-trl) with 🤗 HuggingFace, and we're in the TRL [docs](https://huggingface.co/docs/trl/main/en/sft_trainer#accelerate-fine-tuning-2x-using-unsloth)!
7. `ChatML` for ShareGPT datasets, [conversational notebook](https://colab.research.google.com/drive/1Aau3lgPzeZKQ-98h69CCu1UJcvIBLmy2?usp=sharing)
8. Text completions like novel writing [notebook](https://colab.research.google.com/drive/1ef-tab5bhkvWmBOObepl1WgJvfvSzn5Q?usp=sharing)

<div class="align-center">
  <a href="https://github.com/unslothai/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/u54VK8m8tk"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://ko-fi.com/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Kofi button.png" width="145"></a></a> Support our work if you can! Thanks!
</div>